In [1]:
from __future__ import print_function

import sys
import os
from glob import glob
from time import time
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math
#plt.rcParams['figure.figsize'] = 10,8
import datetime

from   invisible_cities.database import load_db
from   invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.reco.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf
from   invisible_cities.core.core_functions import define_window

import invisible_cities.core.pmaps_functions_c as cpm
from   invisible_cities.core.core_functions import lrange
 
units = SystemOfUnits()
t0 = time()

In [2]:
import S1S2prop as prop
import plotting as plot

## Open 1 Krypton MC file

In [3]:
#%%
mydf_file = os.environ['IC_DATA']+'/pmaps_JM/Kr/pmaps_NEXT_v0_08_09_Kr_ACTIVE_1_0_7bar__10000.root.h5'
print(mydf_file)

mydf = pf.read_pmaps(mydf_file)
#mydf
list(map(type, mydf))
S1df   = mydf[0]
S2df   = mydf[1]
S2Sidf = mydf[2]
print('S1df entries (tbins x events):',len(S1df))
print('S2df entries (tbins x events):',len(S2df))
print('S2Sidf entries:',len(S2Sidf))
type(S1df)
print('Keys of S1df panda dataframe: {} '.format(S1df.keys()))
print('Keys of S2df panda dataframe: {} '.format(S2df.keys()))
print('Keys of S2Sidf panda dataframe: {} '.format(S2Sidf.keys()))

# Convert S12df object  (an S12 pytable read as a PD dataframe) and return an S12L dictionary (list of dict, first dict)
S1dict = pf.df_to_pmaps_dict(S1df,10000)
S2dict = pf.df_to_pmaps_dict(S2df,10000)

evid_S1min = sorted(S1dict.keys())[0]
evid_S1max = sorted(S1dict.keys())[-1]
evid_S2min = sorted(S2dict.keys())[0]
evid_S2max = sorted(S2dict.keys())[-1]
print('First/last event ID (first item in sorted S1 dictionary): {}/{}'.format(evid_S1min,evid_S1max))
print('First/last event ID (first item in sorted S2 dictionary): {}/{}'.format(evid_S2min,evid_S2max))
print('Total number of events in S1 = {}'.format(len(S1dict)))
print('Total number of events in S2 = {}'.format(len(S2dict)))
len(S1dict), type(S1dict), len(S2dict), type(S2dict)


/Users/neus/InvCities/data/pmaps_JM/Kr/pmaps_NEXT_v0_08_09_Kr_ACTIVE_1_0_7bar__10000.root.h5
S1df entries (tbins x events): 51993
S2df entries (tbins x events): 91086
S2Sidf entries: 45115
Keys of S1df panda dataframe: Index(['event', 'evtDaq', 'peak', 'time', 'ene'], dtype='object') 
Keys of S2df panda dataframe: Index(['event', 'evtDaq', 'peak', 'time', 'ene'], dtype='object') 
Keys of S2Sidf panda dataframe: Index(['event', 'evtDaq', 'peak', 'nsipm', 'nsample', 'ene'], dtype='object') 
First/last event ID (first item in sorted S1 dictionary): 0/9999
First/last event ID (first item in sorted S2 dictionary): 0/9999
Total number of events in S1 = 6980
Total number of events in S2 = 9977


(6980, dict, 9977, dict)

In [4]:
length = len(S1dict)
lemax = []
emax       = np.zeros(length, dtype=np.double)
for eventID, EVENTS in S1dict.items():  # loop over dic of events
    for peakID, (ts,Es) in EVENTS.items():
        if(peakID ==1): break
        lemax.append(np.sum(Es))

emax = np.array(lemax)

In [5]:
class S12Prop_new:
    """
    input: S1L
    returns a S1F object for specific peak
    """        
    
    def __init__(self, S12dict):
        self.S12dict = S12dict
        self.length  = len(self.S12dict)
        self.prop()

    def _dict(self):
            return self.S12dict

#    def length(self):
#        return len(self.S12dict)

#test len arrays is == length


    def prop(self):
        self.IDX        = np.zeros(self.length, dtype=np.int) # array index for emax
        self.tIDX       = np.zeros(self.length, dtype=np.double) # time value for IDX
        self.tmin       = np.zeros(self.length, dtype=np.double) # minimum t value
        self.tmax       = np.zeros(self.length, dtype=np.double) # maximum t value
        self.width      = np.zeros(self.length, dtype=np.double) # t width of signal (mus) 
        self.tmean      = np.zeros(self.length, dtype=np.double) # mean time
        self.emin       = np.zeros(self.length, dtype=np.double) # min energy the S1/S2 pulse
        self.emax       = np.zeros(self.length, dtype=np.double) # max energy the S1/S2 pulse
        self.etot       = np.zeros(self.length, dtype=np.double) # total energy
        self.emean      = np.zeros(self.length, dtype=np.double) # total energy
        

        # lists to be filled in the loop. Lists name added 'l': lVARIABLE
        lIDX     = []
        ltIDX    = []
        lemax    = []
        ltmin    = []
        ltmax    = []
        ltmean   = []
        lwidth   = []
        lemin    = []
        lemax    = []
        letot    = []
        lemean   = []

        # 1st loop over dic of events
        # 2nd loop over dic of peaks,(ts,Es)=namedTuple (can also be accessed via value.t, value.E)
        for eventID, EVENTS in self.S12dict.items():  
            for peakID, (ts, Es) in EVENTS.items():  
                if(eventID     == 9999): break  #------> PATCH
                if(peakID        == 1):   break  #------> select one peak 

                lIDX       .append(np.argmax (Es))
                ltIDX      .append(ts[lIDX[-1]]                / units.mus) # use last value of lIDX: [-1]
                ltmin      .append(np.amin(ts))
                ltmax      .append(np.amax(ts))
                lwidth     .append((np.amax(ts) - np.amin(ts)) / units.mus)
                ltmean     .append(np.mean  (ts)               / units.mus)
                lemin      .append(np.amin  (Es))
                lemax      .append(np.amax  (Es))
                letot      .append(np.sum   (Es)) 
                lemean     .append(np.mean  (Es))
                
                
        # convert lists to numpy arrays
        self.IDX      = np.array(lIDX)  
        self.tIDX     = np.array(ltIDX)
        self.tmin     = np.array(ltmin)
        self.tmax     = np.array(ltmax)
        self.width    = np.array(lwidth)
        self.tmean    = np.array(ltmean)
        self.emax     = np.array(lemax)
        self.emin     = np.array(lemin) 
        self.etot     = np.array(letot)
        self.emean    = np.array(lemean)

         
    def S1S2mapd(self, other):
        filt_dict = lambda x, y: dict([ (i , x[i] ) for i in x if i in set(y) ])
      
        keys_S1 = set(self.S12d.keys())
        keys_S2 = set(other.S12d.keys())
        intsect = keys_S1 & keys_S2

        S1map = filt_dict(self.S12d,intsect)
        S2map = filt_dict(other.S12d,intsect)
        return S12Prop(S1map), S12Prop(S2map)
        

In [6]:
S1_new = S12Prop_new(S1dict)

### Get all S1 and S2 in file:  

In [7]:
S1 = prop.S12Prop(S1dict)
S2 = prop.S12Prop(S2dict)

type(S1)
S2.length, S1.length
S1.dict().keys() == S2.dict().keys()

False

In [8]:
S1._dict().keys() == S1_new._dict().keys()

AttributeError: 'S12Prop' object has no attribute '_dict'

In [ ]:
np.array_equal(S1_new.tIDX, np.array(S1.S12Sigt_))

In [ ]:
np.array_equal(S1_new.width, np.array(S1.wS12_))

### Get S1/S2 for events containing 1S1 & 1S2:

In [ ]:
S1map = S1.S1S2mapd(S2)[0]
S2map = S1.S1S2mapd(S2)[1]
S1map._dict().keys() == S2map._dict().keys()

In [ ]:
len(S1map.wS12_)

### S1 features for "all" S1 ("all" = no mapping with S2)

In [ ]:
def multh2(x, nbins, color="red", title="", xlabel="", ylabel="Entries"):
    mycolor = color
#    plt.hist(x, nbins, color = mycolor, histtype="step", alpha=0.75)
    #plt.figure(figsize=(7, 5), dpi=100)

    ## no estamos cogiendo xmin y xmax
    plt.hist(x, nbins, color = mycolor, histtype="bar", alpha=0.75)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    ax = plt.axes()
    ax.tick_params(which='both', direction='in')
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    # add ticks in opposite axis
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    # set labels in corners
    ax.xaxis.set_label_coords(0.9, -0.10)
    ax.yaxis.set_label_coords(-0.1, 0.95)
    
    #axes.xaxis.set_label_coords(0.95, -0.10)
    #axes.yaxis.set_label_coords(-0.1, 0.95)
    return plt

In [ ]:
fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.rc('font', weight='bold')
plt.subplot(221)   
prop.multh(S1.idxt_,    14,  color= "red",title= "", xlabel ="index array for highest phe ", ylabel= "Entries")
plt.subplot(222)   
prop.multh(S1.S12SigE_, 100, color= "red",title= "", xlabel ="highest phe value in peak", ylabel= "Entries")
plt.subplot(223)   
prop.multh(S1.S12Sigt_, 50, color= "red",title= "", xlabel ="time in mus for idxt_ (mus)", ylabel= "Entries")
plt.subplot(224)   
prop.multh(S1.wS12_,    10 , color= "red",title= "", xlabel ="width of signal (mus)", ylabel= "Entries")
plt.show()


In [ ]:
fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.subplot(221)   
prop.multh(S1.tmean_, 50, color= "red",title= "", xlabel ="mean time (mus)", ylabel= "Entries")
plt.subplot(222)   
prop.multh(S1.E_,   100 , color= "red",title= "", xlabel ="total energy (phe)", ylabel= "Entries")
plt.show()

### Obtain drift length:

In [ ]:
S1t = np.array(S1map.S12Sigt_)
S2t = np.array(S2map.S12Sigt_)
z = np.subtract(S2t,S1t)
len(S1t), len(S2t)

### S2 features for "all" S1 ("all" = no mapping with S1)

In [ ]:
fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.rc('font', weight='bold')
plt.subplot(221)   
prop.multh(S2.idxt_,    14,  color= "green",title= "", xlabel ="index array for highest phe ", ylabel= "Entries")
plt.subplot(222)   
prop.multh(S2.S12SigE_, 100, color= "green",title= "", xlabel ="highest phe value in peak", ylabel= "Entries")
plt.subplot(223)   
prop.multh(S2.S12Sigt_, 50, color= "green",title= "", xlabel ="time in mus for idxt_ (mus)", ylabel= "Entries")
plt.subplot(224)   
prop.multh(S2.wS12_,    20 , color= "green",title= "", xlabel ="width of signal (mus)", ylabel= "Entries")
plt.show()



In [ ]:
fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.subplot(221)   
prop.multh(S2.tmean_, 50, color= "green",title= "", xlabel ="mean time (mus)", ylabel= "Entries")
plt.subplot(222)   
prop.multh(S2.E_,   100 , color= "green",title= "", xlabel ="total energy (phe)", ylabel= "Entries")
plt.show()

### Analyze events with S1&S2: (to do: plot it above)

### Obtain drift length:

In [ ]:
S1t = np.array(S1map.S12Sigt_)
S2t = np.array(S2map.S12Sigt_)
z = np.subtract(S2t,S1t)
len(S1t), len(S2t)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.subplot(221)   
plt.hist(z,100,color='black',alpha=0.5)
plt.subplot(222)   
plt.scatter(z,S1map.wS12_)
plt.subplot(223) 
plt.scatter(z,S2map.wS12_)
plt.subplot(224) 
plt.scatter(S1map.wS12_,S2map.wS12_)
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.subplot(221)   
plt.hist(z,100,color='black',alpha=0.5)
plt.subplot(222)   
plt.scatter(z,S1map.wS12_)
plt.subplot(223) 
plt.scatter(z,S2map.wS12_)
plt.subplot(224) 
plt.scatter(S1map.wS12_,S2map.wS12_)
plt.show()

### Save a nice plot

In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

#fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
h1 = prop.h1(S2map.wS12_, 10 , color="black",title="", xlabel ="x (cm)", ylabel = "Events")
h1.show()
#h2 = prop.h1(_S2map.wS12_, 10 ,0., 0.4, color="green",title="", xlabel ="x (cm)", ylabel = "Events")
#h2.show()
#h1.savefig("h2")
#h2.savefig("h2")

In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
import S1S2prop as prop

In [ ]:
#h1 = prop.new_h12(S2map.wS12_, 10 , color="black",title="", xlabel ="x (cm)", ylabel = "Events")
#h1.show()

In [ ]:
def multh(x, nbins, color="red", title="", xlabel="", ylabel="Entries"):
    mycolor = color
#    plt.hist(x, nbins, color = mycolor, histtype="step", alpha=0.75)
    #plt.figure(figsize=(7, 5), dpi=100)

    ## no estamos cogiendo xmin y xmax
    plt.hist(x, nbins, color = mycolor, histtype="bar", alpha=0.75)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    ax = plt.axes()
    ax.tick_params(which='both', direction='in')
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    # add ticks in opposite axis
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    # set labels in corners
    ax.xaxis.set_label_coords(0.9, -0.10)
    ax.yaxis.set_label_coords(-0.1, 0.95)
    
    #axes.xaxis.set_label_coords(0.95, -0.10)
    #axes.yaxis.set_label_coords(-0.1, 0.95)
    return plt


In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FormatStrFormatter

    #           10    2      20     
def new_h12(x, nbins, xmin, xmax, color="red", title="", xlabel="", ylabel="Entries",
       legend="hi"):
    mycolor = color
##    plt.hist(x, nbins, color = mycolor, histtype="step", alpha=0.75)
  #  plt.figure(figsize=(7, 5), dpi=100) 
    plt.hist(x, nbins, color = mycolor, histtype="bar", alpha=0.75)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    # set axis range
    #plt.xlim([xmin,xmax])
    
    majorLocator = MultipleLocator(2)  # x * 2 , subdivisió entre numeros visibles, cuants ticks entre els dos consecutius.
    majorFormatter = FormatStrFormatter('%d')
    minorLocator = MultipleLocator(0.5)  # x value for small ticks

    ax = plt.axes()
    #ax.set_yscale('log')
    
    #ax.xaxis.set_major_locator(majorLocator)
    ax.xaxis.set_major_formatter(majorFormatter)

    
    ## for the minor ticks, use no labels; default NullFormatter
    # ax.xaxis.set_minor_locator(minorLocator)
    ax.tick_params(which='both', direction='in')
    ###plt.rc('font', weight='bold')
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
   
    ax.xaxis.set_label_coords(0.9, -0.10)
    ax.yaxis.set_label_coords(-0.1, 0.95)

    # add ticks in opposite axis
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    
    
   # plt.legend(legend)
    return plt


In [ ]:
h1 = new_h12(S2map.wS12_, 50, 2, 20, color="black",title="", xlabel ="x (cm)", ylabel = "Events")
ax = h1.axes()
ax.set_yscale('log')
h1.show()

In [ ]:
fig = plt.figure(figsize = (10,7))
h2 = multh(S2map.wS12_, 20,  color="black",title="", xlabel ="x (cm)", ylabel = "Events")
ax = h2.axes()
#ax.set_yscale('log')
h2.show()


In [ ]:
min(S2map.E_)

In [ ]:
max(S2map.E_)

In [ ]:
fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
plt.rc('font', weight='bold')
plt.subplot(221)   
prop.multh(S2.idxt_,    14,  color= "green",title= "", xlabel ="index array for highest phe ", ylabel= "Entries")
plt.subplot(222)   
prop.multh(S2.S12SigE_, 100, color= "red",title= "", xlabel ="highest phe value in peak", ylabel= "Entries")
plt.subplot(223)   
prop.multh(S2.S12Sigt_, 50, color= "green",title= "", xlabel ="time in mus for idxt_ (mus)", ylabel= "Entries")
plt.subplot(224)   
prop.multh(S2.wS12_,    20 , color= "green",title= "", xlabel ="width of signal (mus)", ylabel= "Entries")
plt.show()

In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FormatStrFormatter

    #           10    2      20     
def new_h12(x, nbins, xmin, xmax, ax = None, color="red", title="", xlabel="", ylabel="Entries",
       legend="hi"):
    
    
    if ax is None:
        fig, ax = plt.subplots()
        
    mycolor = color
##    plt.hist(x, nbins, color = mycolor, histtype="step", alpha=0.75)
  #  plt.figure(figsize=(7, 5), dpi=100) 
    plt.hist(x, nbins, color = mycolor, histtype="bar", alpha=0.75)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    # set axis range
    #plt.xlim([xmin,xmax])
    
    majorLocator = MultipleLocator(2)  # x * 2 , subdivisió entre numeros visibles, cuants ticks entre els dos consecutius.
    majorFormatter = FormatStrFormatter('%d')
    minorLocator = MultipleLocator(0.5)  # x value for small ticks

    #ax = plt.axes()
    #ax.set_yscale('log')
    
    #ax.xaxis.set_major_locator(majorLocator)
    ax.xaxis.set_major_formatter(majorFormatter)

    
    ## for the minor ticks, use no labels; default NullFormatter
    # ax.xaxis.set_minor_locator(minorLocator)
    ax.tick_params(which='both', direction='in')
    ###plt.rc('font', weight='bold')
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
   
    ax.xaxis.set_label_coords(0.9, -0.10)
    ax.yaxis.set_label_coords(-0.1, 0.95)

    # add ticks in opposite axis
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    
    
   # plt.legend(legend)
    #return plt



In [ ]:
#fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
#plt.rc('font', weight='bold')

fig = plt.figure(figsize = (19,10))
ax1 = fig.add_subplot(2,3,1)
new_h12(S2map.wS12_, 50, 2, 20, ax1, color="black",title="", xlabel ="x (cm)", ylabel = "Events")
ax1.set_yscale('log')
ax2 = fig.add_subplot(2,3,2)
new_h12(S2map.wS12_, 50, 2, 20, ax2, color="blue",title="", xlabel ="x (cm)", ylabel = "Events")
ax3 = fig.add_subplot(2,3,3)
new_h12(S2map.wS12_, 50, 2, 20, ax3, color="green",title="", xlabel ="x (cm)", ylabel = "Events")
ax4 = fig.add_subplot(2,3,4)
new_h12(S2map.wS12_, 50, 2, 20, ax4, color="green",title="", xlabel ="x (cm)", ylabel = "Events")
ax5 = fig.add_subplot(2,3,6)
new_h12(S2map.wS12_, 50, 2, 20, ax5, color="yellow",title="", xlabel ="x (cm)", ylabel = "Events")



In [ ]:
#fig = plt.subplots(nrows = 1, ncols=2, figsize=(15, 10))
#plt.rc('font', weight='bold')

fig = plt.figure(figsize = (25,20))
ax1 = fig.add_subplot(3,3,1)
new_h12(S2map.wS12_, 50, 2, 20, ax1, color="black",title="", xlabel ="x (cm)", ylabel = "Events")
ax1.set_yscale('log')
ax2 = fig.add_subplot(3,3,2)
new_h12(S2map.wS12_, 50, 2, 20, ax2, color="blue",title="", xlabel ="x (cm)", ylabel = "Events")
ax3 = fig.add_subplot(3,3,3)
new_h12(S2map.wS12_, 50, 2, 20, ax3, color="green",title="", xlabel ="x (cm)", ylabel = "Events")
ax4 = fig.add_subplot(3,3,4)
new_h12(S2map.wS12_, 50, 2, 20, ax4, color="green",title="", xlabel ="x (cm)", ylabel = "Events")

ax5 = fig.add_subplot(3,3,9)
new_h12(S2map.wS12_, 50, 2, 20, ax5, color="yellow",title="", xlabel ="x (cm)", ylabel = "Events")


In [ ]:
fig = plt.figure(figsize = (15,8))
ax1 = fig.add_subplot(1,2,1, projection = '3d')
custom_plot1(ax1)
ax2 = fig.add_subplot(1,2,2)
custom_plot2(ax2)

In [ ]:
h1 = new_h12(S2map.wS12_, 50, 2, 20, color="black",title="", xlabel ="x (cm)", ylabel = "Events")
ax = h1.axes()
ax.set_yscale('log')
h1.show()